![nebullvm nebuly AI accelerate inference optimize DeepLearning](https://user-images.githubusercontent.com/38586138/201391643-a80407e5-2c28-409c-90c9-327795cd27e8.png)

# Accelerate ONNX ResNet50 with Speedster

Hi and welcome 👋

In this notebook we will discover how in just a few steps you can speed up the response time of deep learning model inference using the Speedster app from the open-source library `nebullvm`.

We will
1. Install Speedster and the deep learning compilers used by the library.
2. Speed up an ONNX ResNet50 without any loss of accuracy.
3. Achieve faster acceleration on the same model by applying more aggressive optimization techniques (e.g. pruning, quantization) under the constraint of sacrificing up to 2% accuracy.

Let's jump to the code.

In [ ]:
%env CUDA_VISIBLE_DEVICES=0

# Installation

Install Speedster:

In [ ]:
!pip install speedster

Install deep learning compilers:

In [ ]:
!python -m nebullvm.installers.auto_installer --frameworks onnx --compilers all

# Optimization example with ONNX

In the following example we will try to optimize a standard ONNX resnet50.

Speedster can accelerate neural networks without loss of a user-defined precision metric, e.g. accuracy, or can achieve faster acceleration by applying more aggressive optimization techniques, such as pruning and quantization, that may have a negative impact on the selectic metric. The maximum threshold value for accuracy loss is determined by the metric_drop_ths parameter. Read more in the [docs](https://nebuly.gitbook.io/nebuly/nebullvm/get-started).

Let first test the optimization without accuracy loss (metric_drop_ths=0, default value), and then apply further accelerate it under the constrained of losing up to 2% of accuracy (metric = "accuracy", metric_drop_ths = 0.02).

## Scenario 1 - No accuracy drop

First of all we download the pretrained ONNX resnet50 model

In [ ]:
!wget https://github.com/onnx/models/raw/main/vision/classification/resnet/model/resnet50-v1-12.onnx

Then we optimize it with Speedster simple API

In [ ]:
import numpy as np
from speedster import optimize_model, save_model, load_model

# Load a resnet as example
model = "resnet50-v1-12.onnx"

# Provide an input data for the model    
input_data = [((np.random.randn(1, 3, 224, 224).astype(np.float32), ), np.array([0]))]

# Run Speedster optimization
optimized_model = optimize_model(
  model, input_data=input_data, optimization_time="unconstrained"
)

# Try the optimized model
x = np.random.randn(1, 3, 224, 224).astype(np.float32)
res_optimized = optimized_model(x)

We can print the type of the optimized model to see which compiler was faster:

In [ ]:
optimized_model

In our case, the optimized model type was NumpyONNXInferenceLearner, so this means that onnxruntime was the faster compiler.



After the optimization step, we can compare the optimized model with the baseline one in order to verify that the output is the same and to measure the speed improvement

First of all, let's compute and print the original model result



In [ ]:
import onnx
import onnxruntime as ort
from typing import Dict, List


def get_input_names(onnx_model: str):
    model = onnx.load(onnx_model)
    input_all = [node.name for node in model.graph.input]
    return input_all


def get_output_names(onnx_model: str):
    model = onnx.load(onnx_model)
    output_all = [node.name for node in model.graph.output]
    return output_all


def run_onnx_model(
    onnx_model: str, session: ort.InferenceSession, input_tensors: List[np.ndarray], inputs: Dict, output_names: str
) -> List[np.ndarray]:
    
    res = session.run(
        output_names=output_names, input_feed=inputs
    )
    return list(res)


session = ort.InferenceSession(
    model,
    providers=["CUDAExecutionProvider", "CPUExecutionProvider"] # Change to ["CPUExecutionProvider"] if run on cpu
)

inputs = {
    name: array
    for name, array in zip(get_input_names(model), [x])
}

res_original = run_onnx_model(model, session, [x], inputs, get_output_names(model))
res_original

Then, let's print the optimized model result that we computed before

In [ ]:
res_optimized

Then, let's compute the average latency of the baseline model:



In [ ]:
import time

In [ ]:
num_iters = 100

# Warmup
for i in range(10):
  run_onnx_model(model, session, [x], inputs, get_output_names(model))

start = time.time()
for i in range(num_iters):
  run_onnx_model(model, session, [x], inputs, get_output_names(model))
stop = time.time()

print("Average latency original model: {:.4f} seconds".format((stop - start) / num_iters))

Finally we compute the average latency for the optimized model:

In [ ]:
# Warmup
for i in range(10):
  optimized_model(x)

start = time.time()
for i in range(num_iters):
  optimized_model(x)
stop = time.time()

print("Average latency optimized model: {:.4f} seconds".format((stop - start) / num_iters))

## Scenario 2 - Accuracy drop

In this scenario, we set a max threshold for the accuracy drop to 2%

In [ ]:
import numpy as np
from speedster import optimize_model

# Load a resnet as example
model = "resnet50-v1-12.onnx"

# Provide an input data for the model
# Note that in this case we should provide the model at least 100 data samples
input_data = [((np.random.randn(1, 3, 224, 224).astype(np.float32), ), np.array([0])) for i in range(100)]

# Run nebullvm optimization
optimized_model = optimize_model(
  model, input_data=input_data, optimization_time="unconstrained", metric = "accuracy", metric_drop_ths = 0.02
)

# Try the optimized model
x = np.random.randn(1, 3, 224, 224).astype(np.float32)
res_optimized = optimized_model(x)

Here we compute the average throughput for the baseline model:

In [ ]:
num_iters = 100

# Warmup
for i in range(10):
  run_onnx_model(model, session, [x], inputs, get_output_names(model))

start = time.time()
for i in range(num_iters):
  run_onnx_model(model, session, [x], inputs, get_output_names(model))
stop = time.time()

print("Average latency original model: {:.4f} seconds".format((stop - start) / num_iters))

Here we compute the average throughput for the optimized model:

In [ ]:
# Warmup
for i in range(10):
  optimized_model(x)

start = time.time()
for i in range(num_iters):
  optimized_model(x)
stop = time.time()

print("Average latency optimized model: {:.4f} seconds".format((stop - start) / num_iters))

## Save and reload the optimized model

We can easily save to disk the optimized model with the following line:

In [ ]:
save_model(optimized_model, "model_save_path")

We can then load again the model:

In [ ]:
optimized_model = load_model("model_save_path")

<center> 
    <a href="https://discord.com/invite/RbeQMu886J" target="_blank" style="text-decoration: none;"> Join the community </a> |
    <a href="https://nebuly.gitbook.io/nebuly/welcome/questions-and-contributions" target="_blank" style="text-decoration: none;"> Contribute to the library </a>
</center>

<center> 
    <a href="https://github.com/nebuly-ai/nebuly/tree/main/optimization/speedster#key-concepts" target="_blank" style="text-decoration: none;"> How speedster works </a> •
    <a href="https://github.com/nebuly-ai/nebuly/tree/main/optimization/speedster#documentation" target="_blank" style="text-decoration: none;"> Documentation </a> •
    <a href="https://github.com/nebuly-ai/nebuly/tree/main/optimization/speedster#quick-start" target="_blank" style="text-decoration: none;"> Quick start </a> 
</center>